In [ ]:
%matplotlib inline
import tensorflow as tf
import sys  
sys.path.insert(0, './')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam 
from deepexplain.tensorflow import DeepExplain
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
#Local Libraries
from data_manager import DatasetManager
from attention_machine import AttentionNetwork
from feed_forward import FeedForward
from scipy import stats
from sklearn.preprocessing import StandardScaler
from tensorflow.python.ops.parallel_for.gradients import batch_jacobian
try:
        tf.app.flags.DEFINE_string('f', '', 'kernel')
except:
    pass

In [ ]:
dm=DatasetManager(5000,13,version="F4",factor=1)

In [ ]:
Y,data=dm.generate_dataset(display=True)

In [ ]:
x=data[0:data.shape[0]*80//100]
x_test=data[data.shape[0]*80//100:]
y=Y[0:data.shape[0]*80//100,:]
y_test=Y[data.shape[0]*80//100:,:]

In [ ]:
kf = KFold(n_splits=10)
kf.get_n_splits(x)

In [ ]:
def get_anyorder_R_precision(interactions, ground_truth):

    R = len(ground_truth)
    recovered_gt = []
    counter = 0

    for inter, strength in interactions:

        inter_set = set(inter)  # assume 1-indexed
        #print(inter_set)
        #print(ground_truth)
        if any(inter_set <= gt for gt in ground_truth):
            #print("Good")
            
            counter += 1
    R_precision = counter / R

    return R_precision

In [ ]:
def get_pairwise_auc(interactions, ground_truth):
    strengths = []
    gt_binary_list = []
    for inter, strength in interactions:
        inter_set = set(inter)  # assume 1-indexed
        strengths.append(strength)
        #print(inter_set)
        #print(ground_truth)
        if any(inter_set <= gt for gt in ground_truth):
            #print("Good")
            gt_binary_list.append(1)
        else:
            #print("Bad")
            gt_binary_list.append(0)
    print(gt_binary_list)
    auc = roc_auc_score(gt_binary_list, strengths)
    return auc

In [ ]:
run=0
for train_index, val_index in kf.split(x):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]
    np.savetxt('y_trainF4'+str(run)+'.csv',y_train)
    np.savetxt('x_trainF4'+str(run)+'.csv',x_train)
    np.savetxt('y_valF4'+str(run)+'.csv',y_val)
    np.savetxt('x_valF4'+str(run)+'.csv',x_val)
    run+=1

In [ ]:
np.savetxt('y_testF4.csv',y_test)
np.savetxt('x_testF4.csv',x_test)

In [ ]:
run=0
epoch=150
top_num=4

model_layers=[140, 100, 60,20]
results=[]
gt=[{1,2},{3,4},{1,3},{1,4},{2,3},{2,4},{1,5},{2,5},{3,5},{4,5},{6,7},{6,8},{7,8},{9,10}]
top=len(gt)
rankings=[]
x_test=np.genfromtxt("x_testF4.csv")
y_test=np.genfromtxt("y_testF4.csv")
for i in range(10):
    print("Model:"+str(run))
    x_train=np.genfromtxt("x_trainF4"+str(i)+".csv")
    y_train=np.genfromtxt("y_trainF4"+str(i)+".csv")
    x_val=np.genfromtxt("x_valF4"+str(i)+".csv")
    y_val=np.genfromtxt("y_valF4"+str(i)+".csv")
    tf.reset_default_graph()
    #Model
    gamma=0.0000001
    beta=0.0001
    inputs=tf.keras.Input(shape=(x_train.shape[1],))
    network=AttentionNetwork(num_feature=x_train.shape[1],list_reduc=model_layers,l2_strength=beta,dropout_act=False,classification=False)
    p,logit,AX,A,dA_dx,dp_dx,dH_dx=network(inputs)
    model=tf.keras.Model(inputs,p)
    #losses
    def loss_with_custom_regularization(y_true, y_pred):
        cross_entropy=tf.keras.losses.MSE(y_true, y_pred)
        a_regularization=tf.math.scalar_mul(gamma,tf.reduce_sum(tf.abs(A-1)))
        loss_loc=cross_entropy+a_regularization
        return loss_loc

    def loss_classic(y_true, y_pred):
        loss_loc=tf.keras.losses.MSE(y_true, y_pred)
        return loss_loc

    filepath="./attention_machine_non_linearF4"+str(run)+"A.hdf5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',verbose=0, save_best_only=True, mode='min',save_weights_only=True)
    csv=tf.keras.callbacks.CSVLogger('./attention_machine_non_linearF4'+str(run)+'A.log')
    run+=1

    callbacks_list = [checkpoint,csv]
    adam = tf.keras.optimizers.Adam(lr = 0.01)
    model.compile(loss=loss_with_custom_regularization,optimizer=adam,metrics=[loss_classic])
    
    #Train Model
    print("Training...")
    model.fit(x_train, y_train, validation_data=(x_val, y_val),callbacks=callbacks_list, epochs=epoch, batch_size=32,verbose=True)
    
    #get the best model back
    model.load_weights(filepath)
    print("Training done!")
    model.evaluate(x_test, y_test)

    jac_tensor=batch_jacobian(A,inputs,use_pfor=False)
    w=network.logit.get_weights()
    sess=tf.compat.v1.keras.backend.get_session()
    grad_val=jac_tensor.eval(feed_dict={inputs:x_test},session=sess)
    #get interactions
    all_inter=[]
    data_tot=[]
    strengths={}
    for i in range(x_train.shape[1]):
        for j in range(i+1,x_train.shape[1]):
            strengths[str(i+1)+","+str(j+1)]=np.mean(np.abs(w[0][j,0]*grad_val[:,j,i]+w[0][i,0]*grad_val[:,i,j]),axis=0)
    topk_inter=dict(sorted(strengths.items(), key=lambda item: item[1],reverse=True)[:top])
    print(topk_inter)
    dictlist=[]
    rankings.append(dict(sorted(strengths.items(), key=lambda item: item[1],reverse=True)))
    for key, value in topk_inter.items():
        temp = ((int(key.split(",")[0]),int(key.split(",")[1])),value)
        dictlist.append(temp)
    R=get_anyorder_R_precision(dictlist,gt)
    print("R:",R)
    results.append(R)

In [ ]:
mean=np.mean(results)
sd=np.std(results)
print(mean,sd)